# Setting up a Raspberry Pi 4B with YOLOV5 on a 64 bit lite OS

## Scope and overview
The goal of this project is to create a mobile platform for relatively powerful object detection models like the YOLOV5 model used here. Raspberry Pi’s are notoriously minimalistic in its hardware and that philosophy is reflected in their lite operating system versions. For that reason I chose to work on a CLI only 64 bit lite OS on a Raspberry Pi 4B in order to reserve as much computational space as possible for the YOLOV5 model to maximize performance. In order to achieve a visualization on a CLI only operating system I combined YOLOV5 with XVFB framebuffer to render OpenCV's output and a Real VNC server to view the output of XVFB-OpenCV from another Raspberry Pi.

Basic setup:
- 2xRaspberry Pi 4 B 4G RAM/ 64 bit Lite OS
- 32 G Micro SD Card
- Any USB 3.0 or 2.0 camera

## Opencv
Since this is a lite OS a good amount of prerequisite libraries will have to be installed:
```
sudo apt-get install build-essential cmake git unzip pkg-config
sudo apt-get install libjpeg-dev libpng-dev
sudo apt-get install libavcodec-dev libavformat-dev libswscale-dev
sudo apt-get install libgtk2.0-dev libcanberra-gtk* libgtk-3-dev
sudo apt-get install libgstreamer1.0-dev gstreamer1.0-gtk3
sudo apt-get install libgstreamer-plugins-base1.0-dev gstreamer1.0-gl
sudo apt-get install libxvidcore-dev libx264-dev
sudo apt-get install python3-dev python3-numpy python3-pip
sudo apt-get install libtbbmalloc2 libtbb-dev libdc1394-dev
sudo apt-get install libv4l-dev v4l-utils
sudo apt-get install libopenblas-dev libatlas-base-dev libblas-dev
sudo apt-get install liblapack-dev gfortran libhdf5-dev
sudo apt-get install libprotobuf-dev libgoogle-glog-dev libgflags-dev
sudo apt-get install protobuf-compiler
```
Clone the repo from opencv:

```
git clone https://github.com/opencv/opencv.git
```

Before navigating to opencv and building check to see if there is a sufficient amount of memory to conduct the build, its recommended to have at least 5.8 gigabytes of memory available, to check use free -m. If the total memory both RAM and swap are greater than 5.8 you can continue. If not increase the max swap memory with:

```
sudo nano /sbin/dphys-swapfile
```

Find a variable called CONF_MAXSWAP and change it to the memory size in MB. Then set the used swap memory in:

```
Sudo nano /etc/dphys-swapfile
```

And change  CONF_SWAPSIZE to the same memory size

Once there is sufficient memory navigate to opencv, create the build directory, use cmake to configure the build, and build:
```
cd opencv
mkdir build
cd build
cmake ..
make -j$(nproc/2)
```
The build process will take a while but it has been accelerated with swap memory and the nproc/2 argument in make -j.
Note that I decided to use n_cores/2, this is because I ran into some multiprocessing instability issues when utilizing the maximum number of cores on the Pi so only using half of the available cores ensures instability issues wont arise.

Once the build is completed install opencv, create the shared library link, cleanup the build, and update the system:
```
sudo make install
sudo ldconfig
make clean
sudo apt-get update
```

### Compiling optimizations
Since the point of compiling opencv from source is to optimize YOLOV5's performance, there are a lot of optimization compile flags that can be added to make:

- #### Neon || ENABLE_NEON=ON|OFF
  [Neon](https://developer.arm.com/Architectures/Neon) is a SMID instruction set extension for ARM A and R profile processors that allows for parallel computation of multiple values by treating registers as vectors of elements of the same data type.

- #### Vector floating point architecture (VFP) || ENABLE_VFPV3=ON|OFF
  [VFP](https://developer.arm.com/documentation/dui0056/d/using-the-procedure-call-standard/floating-point-options/the-vfp-architecture) is another instruction set extension that handles single or double precision floating point operations while Neon only handles integers and single precision floating point operations. Due to this combined the two provide SMID for single precision floating point vectors and double precise floating point operations.

- #### Thread building blocks (TBB) || WITH_TBB=ON|OFF
  [TBB](http://homepages.math.uic.edu/~jan/mcs572f16/mcs572notes/lec11.html) is a C++ library that contains a high level toolkit for lightweight thread management for parallel processing developed by Intel, to use TBB though you don't need a Intel CPU they developed a ARM version as well.

- #### Open multi-processing (OpenMP) || WITH_OPENMP=ON|OFF
  [OpenMP](https://carleton.ca/rcs/rcdc/introduction-to-openmp/) provides support for [shared memory parallelism](https://rescale.com/glossary/shared-memory-parallelism-smp/) with multithreaded operations.

These optimization arguments are only a few CPU optimizations, for a full comprehensive list of all possible configurations [OpenCV provided documentation on compiling it from source](https://docs.opencv.org/4.x/db/d05/tutorial_config_reference.html?utm_source=chatgpt.com)

## VNC and XVFB

As previously mentioned we are running a 64 bit Lite OS so there is only the command line to work with, XVFB will handle a simulated GUI to allow for OpenCV to operate normally and produce an output window with the YOLOV5 detection output while VNC will handle viewing the XVFB simulated GUI from another Rasberry Pi. First install XVFB and the VNC server on the Raspberry Pi with the lite 64 bit OS:
```
sudo apt install realvnc-vnc-server
sudo apt install xvfb
```
Then create a wireless access point with nmcli:
```
sudo raspi-config
system -> wlan -> set country -> finish

sudo rfkill unblock wifi
sudo ip link set wlan0 up
nmcli con delete SSID_NAME
nmcli con add type wifi ifname wlan0 mode ap con-name SSID_NAME ssid SSID_NAME autoconnect false
nmcli con modify SSID_NAME 802-11-wireless.band bg
nmcli con modify SSID_NAME 802-11-wireless.channel 3
nmcli con modify SSID_NAME 802-11-wireless.cloned-mac-address 00:12:34:56:78:9a
nmcli con modify SSID_NAME ipv4.method shared ipv4.address 192.168.4.1/24
nmcli con modify SSID_NAME ipv6.method disabled
nmcli con modify SSID_NAME wifi-sec.key-mgmt wpa-psk
nmcli con modify SSID_NAME wifi-sec.psk "pass"
nmcli con up SSID_NAME --ask
```
Setup the VNC server:
```
sudo systemctl enable vncserver-x11-serviced
sudo systemctl start vncserver-x11-serviced
export DISPLAY=:0
```
Install and run XVFB:
```
Xvfb :0 -screen 0 1920x1080x24 &
export DISPLAY=:0
```
Finally install and run the VNC client on the second Raspberry Pi with a full 64 bit OS:

```
sudo apt install realvnc-vnc-viewer
realvnc-vnc-server
```

## YOLOV5 
After setting up opencv, it's pretty straightforward from here since all the necessary prerequisite libraries have been installed. Clone YOLOV5 from Ultralytics, navigate to the directory, install the python requirements, and launch the detect script:
```
git clone https://github.com/ultralytics/yolov5.git
cd yolov5
pip3 install -r requirements.txt
python3 detect.py –weights volov5n.pt –img 640 –source 0
```
For this example I am using the YOLOV5 nano weights specified in –weights and a USB camera in –source, since it is the only camera device connected it will be on source 0. Now on the second Raspberry Pi with the 64 bit full OS, connect to the wireless access point:
```
sudo raspi-config
system -> wlan -> set country -> enter SSID -> enter password
```
Then launch the vnc client with `vncviewer`, enter the IPV4 address of the first Raspberry Pi `192.168.4.1` on port `0` `192.168.4.1:0` and the display will appear anchored to the top left. 